In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Работа с данными

Обучение и тестирование модели будет производиться на датасете "CODE-15%: a large scale annotated dataset of 12-lead ECGs". Датасет содержит записи экг, записанные с частотой 400 Гц длительностью 10 секунд в 12-ти отведениях, а также информацию о пациенте: id исследовния, возраст, пол, имеется ли: "1dAVb"(АВ-блокада 1-й степени) \ "RBBB"(блокада правой ветви пучка гиса) \ "LBBB" (блокада левой ветви пучка гиса) \ "SB" (синусовая брадикардия) \
"AF" (есть ли у пациента фибрилляция предсердий или нет) \
"ST" (есть ли у пациента синусовая тахикардия или нет); id пациента, нормальная ли ЭКГ, умер ли пациент и время смерти, если да. Данные представлены в виде таблицы в формате .csv. На часть из них можно посмотреть ниже.

**Данные о пациентах**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
exams_path = "/content/drive/MyDrive/exams.csv"
information = pd.read_csv(exams_path)

In [4]:
information.head(10)

,exam_id,age,is_male,nn_predicted_age,1dAVb,RBBB,LBBB,SB,ST,AF,patient_id,death,timey,normal_ecg,trace_file
0,1169160,38,True,40.160484,False,False,False,False,False,False,523632,False,2.098628,True,exams_part13.hdf5
1,2873686,73,True,67.059440,False,False,False,False,False,False,1724173,False,6.657529,False,exams_part13.hdf5
2,168405,67,True,79.621740,False,False,False,False,False,True,51421,False,4.282188,False,exams_part13.hdf5
3,271011,41,True,69.750260,False,False,False,False,False,False,1737282,False,4.038353,True,exams_part13.hdf5
4,384368,73,True,78.873460,False,False,False,False,False,False,331652,False,3.786298,False,exams_part13.hdf5
5,2950575,61,True,70.905174,False,False,False,False,False,False,17423,NaN,NaN,False,exams_part13.hdf5
6,1467619,33,False,48.628563,False,False,False,False,False,False,1351337,False,1.498629,False,exams_part13.hdf5
7,1537328,24,True,29.179337,False,False,False,False,False,False,1519774,False,1.367122,True,exams_part13.hdf5
8,981735,82,True,82.224610,False,False,False,False,False,False,1192754,False,2.468491,False,exams_part13.hdf5
9,132538,85,False,87.512890,False,False,False,False,False,True,60291,NaN,NaN,False,exams_part13.hdf5


**Сигналы ЭКГ**

Записи экг представлены в виде 3 мерного тензора (N, 4096, 12), где N - это количество исследований, 4096 - количество записанных сигналов, 12 - количество отведений.

Загружаем записи экг.

In [5]:
path_to_signals0 = '/content/drive/MyDrive/exams_part0.hdf5'
path_to_signals1 = '/content/drive/MyDrive/exams_part1.hdf5'
path_to_signals2 = '/content/drive/MyDrive/exams_part2.hdf5'
path_to_signals3 = '/content/drive/MyDrive/exams_part3.hdf5'
path_to_signals4 = '/content/drive/MyDrive/exams_part4.hdf5'
signals_folder0 = h5py.File(path_to_signals0, 'r')
signals_folder1 = h5py.File(path_to_signals1, 'r')
signals_folder2 = h5py.File(path_to_signals2, 'r')
signals_folder3 = h5py.File(path_to_signals3, 'r')
signals_folder4 = h5py.File(path_to_signals4, 'r')

In [6]:
list(signals_folder0.keys())

['exam_id', 'tracings']

Каждая папка содержит 2 массива: 1) id исследований (для связи с пациентом); 2) тензор самих сигналов (N, 4096, 12).

In [7]:
exam_id0 = np.array(signals_folder0['exam_id'])
exam_id0 = exam_id0[0:20000]
print("Количество исследований в папке №0:", exam_id0.shape)

exam_id1 = np.array(signals_folder1['exam_id'])
exam_id1 = exam_id1[0:20000]
print("Количество исследований в папке №1:", exam_id1.shape)

exam_id2 = np.array(signals_folder2['exam_id'])
exam_id2 = exam_id2[0:20000]
print("Количество исследований в папке №2:", exam_id2.shape)

exam_id3 = np.array(signals_folder3['exam_id'])
exam_id3 = exam_id3[0:20000]
print("Количество исследований в папке №3:",exam_id3.shape)

exam_id4 = np.array(signals_folder4['exam_id'])
exam_id4 = exam_id4[0:20000]
print("Количество исследований в папке №4:",exam_id3.shape)

print("№0:", exam_id0)
print("№1:", exam_id1)
print("№2:", exam_id2)
print("№3:", exam_id3)
print("№4:", exam_id4)

Количество исследований в папке №0: (20000,)
Количество исследований в папке №1: (20000,)
Количество исследований в папке №2: (20000,)
Количество исследований в папке №3: (20000,)
Количество исследований в папке №4: (20000,)
№0: [ 590673  214626 2936711 ... 1444004 2659750  536312]
№1: [4409337  835969 2540049 ...  523675  880632 2872039]
№2: [  18903 2902407 4274913 ... 2866962 2661384 1179015]
№3: [3190119  683626 2767061 ...  220811 2795569 1450162]
№4: [ 920511  379941 3053027 ...  891207 3048377 1778106]


In [8]:
exam_id = np.hstack([exam_id0, exam_id1, exam_id2, exam_id3, exam_id4])
print(exam_id.shape)
print(exam_id)

(100000,)
[ 590673  214626 2936711 ...  891207 3048377 1778106]


Каждому исследованию сопоставим его порядковый номер

In [9]:
id_dict = {}
for i in range (0,345779):
  id_dict[information.exam_id[i]] = i


Сопоставим каждому исследованию вектор размерности 6 из 0 и 1. 1 означает, что данная болезнь есть у пациента, a 0 - нет.

In [16]:
results = np.eye(100000, 6)

for i in range(0,100000):
  ind = id_dict[exam_id[i]]

  if information['1dAVb'][ind]:
    results[i][0] = 1
  else:
    results[i][0] = 0

  if information['RBBB'][ind]:
    results[i][1] = 1
  else:
    results[i][1] = 0

  if information['LBBB'][ind]:
    results[i][2] = 1
  else:
    results[i][2] = 0

  if information['SB'][ind]:
    results[i][3] = 1
  else:
    results[i][3] = 0

  if information['ST'][ind]:
    results[i][4] = 1
  else:
    results[i][4] = 0

  if information['AF'][ind]:
    results[i][5] = 1
  else:
    results[i][5] = 0

print(results[150:160,:])

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


Теперь работаем с сигналами экг. Датасет содержит 10-секундные и 7-секундные записи. Если запись имеет длительность 7 секунд, то она дополнена в начале и в конце 648 нулями. Возьмём от всех записей средние 7 секунд, т.е. от 648 до 3448 сигнала.

In [19]:
signals0 = np.array(signals_folder0['tracings'][:,648:3448,:20000])

Преобразуем записи ЭКГ в спектрограммы и сохраняем новые данные.

In [22]:
def get_spectrogram(ecg_signal):
   spectrogram = tf.signal.stft(ecg_signal, frame_length=256, frame_step=128)
   spectrogram = tf.abs(spectrogram**2)
   return spectrogram

In [23]:
def spectrogram_dataset(dataset, results):
  spec_signals = []
  spec_results = []
  count = 0
  for i in range (0, results.shape[0]):
    if sum(results[i]) == 0 and count < 3000:
      count+=1
      mas = []
      for lead in range (0,12):
        mas.append(np.array(get_spectrogram(dataset[i,:,lead])))
      mas1 = np.concatenate([mas[0][..., np.newaxis],mas[1][..., np.newaxis],mas[2][..., np.newaxis]
                           ,mas[3][..., np.newaxis],mas[4][..., np.newaxis],mas[5][..., np.newaxis],mas[6][..., np.newaxis],
                           mas[7][..., np.newaxis],mas[8][..., np.newaxis],mas[9][..., np.newaxis],mas[10][..., np.newaxis],
                           mas[11][..., np.newaxis]], axis = -1)
      spec_signals.append(mas1)
      spec_results.append(results[i])

    if sum(results[i]) > 0 and count:
      mas = []
      for lead in range (0,12):
        mas.append(np.array(get_spectrogram(dataset[i,:,lead])))

      mas1 = np.concatenate([mas[0][..., np.newaxis],mas[1][..., np.newaxis],mas[2][..., np.newaxis]
                           ,mas[3][..., np.newaxis],mas[4][..., np.newaxis],mas[5][..., np.newaxis],mas[6][..., np.newaxis],
                           mas[7][..., np.newaxis],mas[8][..., np.newaxis],mas[9][..., np.newaxis],mas[10][..., np.newaxis],
                           mas[11][..., np.newaxis]], axis = -1)
      spec_signals.append(mas1)
      spec_results.append(results[i])
  return np.array(spec_signals), np.array(spec_results)

In [24]:
data, result = spectrogram_dataset(signals0,results[0:20000])

In [25]:
data.shape

(5184, 20, 129, 12)

In [26]:
result.shape

(5184, 6)

In [27]:
np.save('spec_data0.npy', data)
np.save('spec_res0.npy', result)